In [1]:
import pandas as pd
import numpy as np


from Project.Database import Db
from Project._03FeatureSelection.NZERTF_dataframe_redundancy_functions import find_average_power_consumption
from Project._05InferKnowledgeOfRules.infer_rules_functions import SE_time_df, json_to_dataframe

In [2]:
app_stats = SE_time_df(json_to_dataframe(year=2, level=1, with_redundancy=False))

In [3]:
app_stats['Load_StatusApplianceDishwasher']

,TotalAbsSupport,AbsSupport,EventCount,ExternalUtility,RelSupport,TimeAssociation,TimespanMean,TimespanMin,TimespanMax,Flexibility
0,0,0,260,0.712329,0.000000,0,131.919892,129.983333,133.0,1.804511
1,0,0,260,0.712329,0.000000,0,131.919892,129.983333,133.0,1.804511
2,0,0,260,0.712329,0.000000,0,131.919892,129.983333,133.0,1.804511
3,0,0,260,0.712329,0.000000,0,131.919892,129.983333,133.0,1.804511
4,0,0,260,0.712329,0.000000,0,131.919892,129.983333,133.0,1.804511
5,0,0,260,0.712329,0.000000,0,131.919892,129.983333,133.0,1.804511
6,0,0,260,0.712329,0.000000,0,131.919892,129.983333,133.0,1.804511
7,0,0,260,0.712329,0.000000,0,131.919892,129.983333,133.0,1.804511
8,0,0,260,0.712329,0.000000,0,131.919892,129.983333,133.0,1.804511
9,0,0,260,0.712329,0.000000,0,131.919892,129.983333,133.0,1.804511


In [4]:
average_power_consumption_df = find_average_power_consumption()
average_power_consumption_df

Load_StatusApplianceCooktop             1496.0
Load_StatusApplianceDishwasher           282.0
Load_StatusApplianceOven                 771.0
Load_StatusApplianceRangeHood             78.0
Load_StatusBA1Lights                      48.0
Load_StatusBA2Lights                      11.0
Load_StatusBR2Lights                      40.0
Load_StatusBR3Lights                      33.0
Load_StatusBR4Lights                      27.0
Load_StatusDRLights                       38.0
Load_StatusEntryHallLights                16.0
Load_StatusKitchenLightsA                 28.0
Load_StatusKitchenLightsB                 33.0
Load_StatusKitchenLightsC                 28.0
Load_StatusLRLights1                      18.0
Load_StatusLRLights2                      41.0
Load_StatusLRLights3                      27.0
Load_StatusLatentload                     45.0
Load_StatusMBALights                      84.0
Load_StatusMBRLights1                     25.0
Load_StatusMBRLights2                     25.0
Load_StatusPl

In [5]:
average_power_consumption_df[['Load_StatusApplianceDishwasher', 'Load_StatusPlugLoadVacuum', 'Load_StatusClothesWasher', 'Load_StatusDryerPowerTotal', 'Load_StatusPlugLoadIron']]

Load_StatusApplianceDishwasher     282.0
Load_StatusPlugLoadVacuum          583.0
Load_StatusClothesWasher           168.0
Load_StatusDryerPowerTotal        3426.0
Load_StatusPlugLoadIron           1303.0
dtype: float64

In [6]:
average_power_consumption_df.loc[~average_power_consumption_df.index.isin(app_stats.keys())]

Load_StatusBR4Lights                      27.0
Load_StatusEntryHallLights                16.0
Load_StatusLRLights1                      18.0
Load_StatusLatentload                     45.0
Load_StatusPlugLoadHairDryerCurlIron    1515.0
dtype: float64

In [7]:
matrix_dict={}
for key, value in app_stats.items():
    timespan = value['TimespanMax'][0]
    hour_slots = int(np.ceil(timespan/60))

    average_power_consumption = average_power_consumption_df.loc[lambda self: self.index == key].values[0]

    hour_list = [0 for n in range(24)]
    for i in range(hour_slots):
        hour_list[i] = round(average_power_consumption*timespan/60_000/hour_slots, 4)

    temp_dict={}
    for n in range(24+1-hour_slots):
        rotate_list = [hour_list[(i - n) % len(hour_list)] for i, x in enumerate(hour_list)]
        temp_dict.update({n: rotate_list})

    matrix_dict.update({key: pd.DataFrame(temp_dict).loc[:,lambda self: self.sum(0) == self.T.dot(app_stats[key]['TimeAssociation'])]})

matrix_dict

{'Load_StatusSensHeatPrntAUP': Empty DataFrame
 Columns: []
 Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
 'Load_StatusSensHeatPrntBUP':          0       1
 0   0.0632  0.0000
 1   0.0632  0.0632
 2   0.0632  0.0632
 3   0.0632  0.0632
 4   0.0632  0.0632
 5   0.0632  0.0632
 6   0.0632  0.0632
 7   0.0632  0.0632
 8   0.0632  0.0632
 9   0.0632  0.0632
 10  0.0000  0.0632
 11  0.0000  0.0000
 12  0.0000  0.0000
 13  0.0000  0.0000
 14  0.0000  0.0000
 15  0.0000  0.0000
 16  0.0000  0.0000
 17  0.0000  0.0000
 18  0.0000  0.0000
 19  0.0000  0.0000
 20  0.0000  0.0000
 21  0.0000  0.0000
 22  0.0000  0.0000
 23  0.0000  0.0000,
 'Load_StatusSensHeatChildAUP':          0       1       2       3
 0   0.0694  0.0000  0.0000  0.0000
 1   0.0694  0.0694  0.0000  0.0000
 2   0.0694  0.0694  0.0694  0.0000
 3   0.0694  0.0694  0.0694  0.0694
 4   0.0694  0.0694  0.0694  0.0694
 5   0.0694  0.0694  0.0694  0.0694
 6   0.0694  0.0694  0.0694  0

In [8]:
production = Db.load_data(consumption=False, production=True, year=2)

In [9]:
production_vectors = production['CO2(g/Wh)'].groupby([production.index.strftime('%Y-%m-%d'), production.index.hour]).sum()

In [10]:
production_vectors

2015-02-01  0     0.734305
            1     0.752407
            2     0.737095
            3     0.733298
            4     0.731043
                    ...   
2016-01-31  19     0.68451
            20    0.697113
            21    0.544998
            22    0.420529
            23    0.420529
Name: CO2(g/Wh), Length: 8760, dtype: object

In [11]:
production_vectors.loc[production_vectors.index.get_level_values(level=0).unique().tolist()[1]]

0     0.520168
1     0.687219
2     0.377188
3     0.444097
4     0.618123
5     0.714707
6     0.632088
7     0.793935
8     0.812028
9     0.748806
10    0.744907
11    0.739977
12    0.710937
13    0.686382
14     0.72136
15    0.710937
16    0.787372
17    0.741221
18    0.765063
19    0.774067
20    0.746728
21    0.769018
22    0.892077
23    0.837508
Name: CO2(g/Wh), dtype: object

In [12]:
pd.to_datetime(production_vectors.index.get_level_values(level=0).unique(), format='%Y-%m-%d').weekday.map(lambda self: {0: 'Monday',
                   1: 'Tuesday',
                   2: 'Wednesday',
                   3: 'Thursday',
                   4: 'Friday',
                   5: 'Saturday',
                   6: 'Sunday',}[self])

Index(['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',
       'Saturday', 'Sunday', 'Monday', 'Tuesday',
       ...
       'Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday', 'Friday', 'Saturday', 'Sunday'],
      dtype='object', length=365)

In [13]:
def production_vector(day_number: int=0):
    return production_vectors.loc[production_vectors.index.get_level_values(level=0).unique().tolist()[day_number]]

In [14]:
def find_min_hour(appliance: str, day_number: int, number_of_events: int):
    energy_matrix = matrix_dict[appliance]
    emission_vector = production_vector(day_number)
    return_df = pd.DataFrame()
    if number_of_events == 1:
        event = energy_matrix.T.dot(emission_vector)[lambda self: self == self.min()].head(1)
        index = event.index.tolist()[0]
        return_df[index] = energy_matrix[index].multiply(emission_vector)
        return return_df
    elif number_of_events > 1:
        events = energy_matrix.T.dot(emission_vector)[lambda self: self == self.min()].head(1)

        condition = (lambda df:
                     (df.sum(0) != df.T.dot(np.where(pd.DataFrame(df[events.index.tolist()]).max(1) > 0, 1, 0))))
        condition2 = (lambda df, events_index:
                      (df.loc[events_index] == 0).min(0))

        for _ in range(number_of_events-1):
            events = pd.concat(objs=(
                events,
                energy_matrix.loc[:,
               (lambda self:
                condition(self) &
                condition2(self, events.index.tolist()))].T.dot(emission_vector)[lambda self: self == self.min()].head(1)
            )
            )

        for index in events.index:
            return_df[index] = energy_matrix[index].multiply(emission_vector)
        return return_df

In [15]:
find_min_hour('Load_StatusApplianceDishwasher', day_number=0, number_of_events=1)

,21
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


In [16]:
find_min_hour('Load_StatusDryerPowerTotal', day_number=0, number_of_events=3)

,21,20,12
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
5,0.0,0.0,0.0
6,0.0,0.0,0.0
7,0.0,0.0,0.0
8,0.0,0.0,0.0
9,0.0,0.0,0.0


In [17]:
year2, meta = Db.load_data(consumption=True, year=2, meta=True, hourly=False)

In [18]:
meta.loc['Load_StatusPlugLoadIron']

Subsystem                                                                      Loads
Measurement_Location                                                   MasterBedroom
Parameter                                                               Status_OnOff
Description             Number to indicate whether iron is activated (1: Yes, O: No)
Units                                                                   BinaryStatus
Aggregation_Method                                                           Average
min_value                                                                        0.0
max_value                                                                        1.0
Standby_Power                                                                    0.0
Measurement_Floor                                                           2ndFloor
Load_Match                                               Load_MBRPlugLoadsPowerUsage
Status_Match                                                     

In [19]:
appliance_condition = (lambda self: (
        (self["Parameter"] == "Status_OnOff") &
        (self["Subsystem"] == "Loads") &
        # (~self.index.str.contains("Child")) &
        # (~self.index.str.contains("Prnt")) &
        (~self.index.str.contains("StatusLatentload"))  #&
    # (~self.index.str.contains("Load_StatusClothesWasher")) &
    # (~self.index.str.contains("Load_StatusDryerPowerTotal"))
))

appliances_statusOnOff = meta.loc[appliance_condition].index.tolist()

In [20]:
minute_appliances_status = year2[["Timestamp"] + appliances_statusOnOff].copy()

In [21]:
movable_appliances = ['Load_StatusApplianceDishwasher', 'Load_StatusPlugLoadVacuum', 'Load_StatusClothesWasher', 'Load_StatusDryerPowerTotal', 'Load_StatusPlugLoadIron']

In [22]:
minute_appliances_status.groupby(minute_appliances_status['Timestamp'].dt.strftime('%Y-%m-%d')).max()[movable_appliances]

,Load_StatusApplianceDishwasher,Load_StatusPlugLoadVacuum,Load_StatusClothesWasher,Load_StatusDryerPowerTotal,Load_StatusPlugLoadIron
Timestamp,,,,,
2015-02-01,1,0,1,1,1
2015-02-02,1,0,0,0,0
2015-02-03,0,0,0,0,0
2015-02-04,1,0,1,1,0
2015-02-05,0,0,0,0,0
2015-02-06,1,0,0,0,0
2015-02-07,1,1,1,1,1
2015-02-08,1,0,1,1,1
2015-02-09,1,0,0,0,0


In [23]:
level1_json_df = json_to_dataframe(year=2, level=1, exclude_follows=True, with_redundancy=False)

In [24]:
level1_movable_appliances = level1_json_df.loc[lambda self: self['pattern'].isin(movable_appliances)].set_index('pattern')

In [25]:
level1_movable_appliances

,supp,conf,time,rule,multi_floor,floor
pattern,,,,,,
Load_StatusClothesWasher,0.427397,1.0,"{'0': [['2015-02-01 11:00:49', '2015-02-01 11:57:49'], ['2015-02-01 14:30:49', '2015-02-01 15:28:48']], '3': [['2015-02-04 18:30:22', '2015-02-04 19:27:22'], ['2015-02-04 20:25:22', '2015-02-04 21...",app,False,1stFloor
Load_StatusDryerPowerTotal,0.432877,1.0,"{'0': [['2015-02-01 12:30:49', '2015-02-01 13:03:49'], ['2015-02-01 15:00:48', '2015-02-01 15:28:48']], '3': [['2015-02-04 20:30:22', '2015-02-04 20:58:22']], '6': [['2015-02-07 12:31:00', '2015-0...",app,False,1stFloor
Load_StatusPlugLoadIron,0.287671,1.0,"{'0': [['2015-02-01 14:15:49', '2015-02-01 14:29:49']], '6': [['2015-02-07 18:00:00', '2015-02-07 18:24:00']], '7': [['2015-02-08 14:15:52', '2015-02-08 14:29:52']], '13': [['2015-02-14 17:59:10',...",app,False,2ndFloor
Load_StatusApplianceDishwasher,0.712329,1.0,"{'0': [['2015-02-01 20:27:48', '2015-02-01 22:39:48']], '1': [['2015-02-02 20:27:40', '2015-02-02 22:39:40']], '3': [['2015-02-04 20:28:22', '2015-02-04 22:40:22']], '5': [['2015-02-06 20:15:05', ...",app,False,1stFloor
Load_StatusPlugLoadVacuum,0.142466,1.0,"{'6': [['2015-02-07 17:30:00', '2015-02-07 19:21:00']], '13': [['2015-02-14 17:29:10', '2015-02-14 19:19:10']], '20': [['2015-02-21 17:29:28', '2015-02-21 19:20:28']], '27': [['2015-02-28 17:29:38...",app,False,1stFloor


In [26]:
def find_event_count(appliance: str, day_number: int=0):
    try:
        return len(level1_movable_appliances.loc[appliance]['time'][f'{day_number}'])
    except:
        return 0

In [27]:
def hourly_dataframe(df, aggregate_func: str):
    df['Day'] = df['Timestamp'].dt.dayofyear
    df['Day'] = df['Day'] - df['Day'][0]
    # Find the first row index with negative value
    first_index = df.loc[lambda self: self['Day'] < 0].index.tolist()[0]
    df.loc[first_index:, 'Day'] = df.loc[first_index:, 'Day'] + 365

    df['Hour'] = df['Timestamp'].dt.hour
    df['Timestamp'] = pd.to_datetime(df['Timestamp'].dt.strftime('%Y-%m-%d %H'), format='%Y-%m-%d %H')
    if aggregate_func == 'max':
        df = df.groupby(['Day', 'Hour']).max()
    elif aggregate_func == 'sum':
        df = df.groupby(['Day', 'Hour']).sum()
    df.reset_index(inplace=True)
    df['Emission'] = 0
    return df

In [28]:
year2 = Db.load_data(year=2, hourly=False)[['Timestamp'] + movable_appliances]

year2 = hourly_dataframe(year2, aggregate_func='max')

In [29]:
year2_optim = year2.copy()
year2_optim[movable_appliances] = 0
for app in movable_appliances:
    for day in year2_optim['Day'].unique():
        event_count = find_event_count(appliance=app, day_number=day)
        if event_count > 0:
            events = find_min_hour(appliance=app, day_number=day, number_of_events=event_count)
            for column in events.columns:
                for index, emission in events.loc[lambda self: self[column] > 0, column].iteritems():
                    year2_optim.loc[lambda self: (self['Day'] == day) & (self['Hour'] == index), 'Emission'] += emission
                    year2_optim.loc[lambda self: (self['Day'] == day) & (self['Hour'] == index), app] = 1

year2_optim

,Day,Hour,Timestamp,Load_StatusApplianceDishwasher,Load_StatusPlugLoadVacuum,Load_StatusClothesWasher,Load_StatusDryerPowerTotal,Load_StatusPlugLoadIron,Emission
0,0,0,2015-02-01 00:00:00,0,0,0,0,0,0.000000
1,0,1,2015-02-01 01:00:00,0,0,0,0,0,0.000000
2,0,2,2015-02-01 02:00:00,0,0,0,0,0,0.000000
3,0,3,2015-02-01 03:00:00,0,0,0,0,0,0.000000
4,0,4,2015-02-01 04:00:00,0,0,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...
8667,364,19,2016-01-31 19:00:00,0,0,0,0,0,0.000000
8668,364,20,2016-01-31 20:00:00,0,0,0,0,0,0.000000
8669,364,21,2016-01-31 21:00:00,1,0,0,1,0,1.140516
8670,364,22,2016-01-31 22:00:00,1,0,0,0,0,0.087638


In [30]:
year2_optim['Emission'].sum()

549.4404883704052

In [31]:
year2_orig = year2.copy()

for app in movable_appliances:
    lifespan = app_stats[app]['TimespanMax'][0]
    hour_slots = int(np.ceil(lifespan/60))
    average_power_consumption = average_power_consumption_df.loc[lambda self: self.index == app].values[0]
    energy_consumption_per_hour = round(average_power_consumption*lifespan/60_000/hour_slots, 4)

    for day in year2_orig['Day'].unique():
        emission_vector = production_vector(day)
        energy_vector = year2_orig.loc[lambda self: self['Day'] == day, app].multiply(energy_consumption_per_hour).reset_index(drop=True)
        energy_consumed = energy_vector.multiply(emission_vector)
        for index, emission in energy_consumed[lambda self: self > 0].iteritems():
            year2_orig.loc[lambda self: (self['Day'] == day) & (self['Hour'] == index), 'Emission'] += emission


year2_orig

,Day,Hour,Timestamp,Load_StatusApplianceDishwasher,Load_StatusPlugLoadVacuum,Load_StatusClothesWasher,Load_StatusDryerPowerTotal,Load_StatusPlugLoadIron,Emission
0,0,0,2015-02-01 00:00:00,0,0,0,0,0,0.000000
1,0,1,2015-02-01 01:00:00,0,0,0,0,0,0.000000
2,0,2,2015-02-01 02:00:00,0,0,0,0,0,0.000000
3,0,3,2015-02-01 03:00:00,0,0,0,0,0,0.000000
4,0,4,2015-02-01 04:00:00,0,0,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...
8667,364,19,2016-01-31 19:00:00,0,0,0,0,0,0.000000
8668,364,20,2016-01-31 20:00:00,1,0,0,0,0,0.145278
8669,364,21,2016-01-31 21:00:00,1,0,0,0,0,0.113577
8670,364,22,2016-01-31 22:00:00,1,0,0,0,0,0.087638


In [32]:
meta = Db.load_data(meta=True, hourly=False, year=2, consumption=False).loc[lambda self: (~self['Consumer_Match'].isna()), 'Consumer_Match']

year2_with_redun = Db.load_data(year=2, hourly=False)[['Timestamp'] + meta.tolist()]
year2_with_redun = hourly_dataframe(year2_with_redun, aggregate_func='sum')
year2_with_redun['Consumption'] = year2_with_redun[meta.tolist()].sum(1).div(60_000)
year2_with_redun.drop(meta.tolist(), axis=1, inplace=True)

year2_no_redun = Db.load_data(year=2, hourly=False, with_redundancy=False)[['Timestamp'] + meta.tolist()]
year2_no_redun = hourly_dataframe(year2_no_redun, aggregate_func='sum')
year2_no_redun['Consumption'] = year2_no_redun[meta.tolist()].sum(1).div(60_000)
year2_no_redun.drop(meta.tolist(), axis=1, inplace=True)


In [33]:
def find_emissions(df, emission_vec):
    energy_vec = df.loc[lambda self: self['Day'] == day, 'Consumption'].reset_index(drop=True)
    energy_consumed = energy_vec.multiply(emission_vec)
    for index, emission in energy_consumed[lambda self: self > 0].iteritems():
        df.loc[lambda self: (self['Day'] == day) & (self['Hour'] == index), 'Emission'] += emission
    return df


In [34]:
for day in year2_with_redun['Day'].unique():
    emission_vector = production_vector(day)
    year2_with_redun = find_emissions(year2_with_redun, emission_vec=emission_vector)
    year2_no_redun = find_emissions(year2_no_redun, emission_vec=emission_vector)

In [35]:
year2_with_redun

,Day,Hour,Emission,Consumption
0,0,0,1.305913,1.778433
1,0,1,1.577106,2.096081
2,0,2,1.357881,1.842206
3,0,3,1.338302,1.825044
4,0,4,1.347818,1.843692
...,...,...,...,...
8667,364,19,6.204622,9.064332
8668,364,20,5.643320,8.095270
8669,364,21,4.622138,8.481025
8670,364,22,4.098887,9.746986


In [36]:
year2_no_redun

,Day,Hour,Emission,Consumption
0,0,0,1.305913,1.778433
1,0,1,1.577106,2.096081
2,0,2,1.357881,1.842206
3,0,3,1.338302,1.825044
4,0,4,1.347818,1.843692
...,...,...,...,...
8667,364,19,5.280107,7.713708
8668,364,20,5.218687,7.486140
8669,364,21,4.622138,8.481025
8670,364,22,4.060225,9.655049


In [37]:
year2_redun_emission = year2_with_redun.groupby(['Day']).sum()['Emission'].sum()

year2_no_redun_emission = year2_no_redun.groupby(['Day']).sum()['Emission'].sum()


In [38]:
year2_redun_emission

26556.42690101065

In [39]:
year2_no_redun_emission

25942.586521546084

In [40]:
redun_diff_emission=year2_redun_emission-year2_no_redun_emission
redun_diff_emission

613.8403794645674

In [41]:
redun_diff_emission/year2_redun_emission*100

2.3114569657757937

In [42]:
year2_no_redun

,Day,Hour,Emission,Consumption
0,0,0,1.305913,1.778433
1,0,1,1.577106,2.096081
2,0,2,1.357881,1.842206
3,0,3,1.338302,1.825044
4,0,4,1.347818,1.843692
...,...,...,...,...
8667,364,19,5.280107,7.713708
8668,364,20,5.218687,7.486140
8669,364,21,4.622138,8.481025
8670,364,22,4.060225,9.655049


In [43]:
redun_diff_emission

613.8403794645674